In [7]:
import torch
import os

os.chdir("c:/Users/cunn2/OneDrive/DSML/Project/thesis-repo")

from sms.exp1.config_classes import load_config_from_launchplan
from sms.exp1.run_training import build_encoder, build_projector
from sms.exp1.models.siamese import SiameseModel

config = load_config_from_launchplan("sms/exp1/runs/transformer_rel_big_1/original_launchplan.yaml")

encoder = build_encoder(config.model_dump())
projector = build_projector(config.model_dump())

model = SiameseModel(encoder, projector)

print(encoder)
print(projector)
print(model)

pt_encoder = build_encoder(config.model_dump())
pt_projector = build_projector(config.model_dump())
pt_model = SiameseModel(pt_encoder, pt_projector)   
pt_model.load_state_dict(torch.load("sms/exp1/runs/transformer_rel_big_1/pretrain_saved_model.pth"))    

ft_encoder = build_encoder(config.model_dump())
ft_encoder.load_state_dict(torch.load("sms/exp1/runs/transformer_rel_big_1/finetune_saved_model.pth"))

BertEncoder(
  (emb): TokenAndPositionalEmbeddingLayer(
    (token_emb): Conv1d(2, 128, kernel_size=(1,), stride=(1,))
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=128, out_features=256, bias=True)
  (pool): AdaptiveAvgPool1d(output_size=1)
)
ProjectionHead(
  (projector): Sequential(
    (0): Li

C:\Users\cunn2\AppData\Local\Temp\ipykernel_127412\2110709451.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pt_model.load_state_dict(torch.load("sms/exp1/runs/transfo

<All keys matched successfully>

In [3]:
from sms.src.synthetic_data.formatter import InputFormatter
from sms.src.synthetic_data.note_arr_mod import NoteArrayModifier
import numpy as np
import logging
from sms.src.log import configure_logging

logger = logging.getLogger(__name__)
configure_logging(console_level=logging.INFO)

data = torch.load(r"C:\Users\cunn2\OneDrive\DSML\Project\thesis-repo\data\exp1\train_data.pt")

formatter = InputFormatter(**config.model_dump()['input'])

aug_dict = {
    "use_transposition": False,
    "use_shift_selected_notes_pitch": False,
    "use_change_note_durations": True,
    "use_delete_notes": False,
    "use_insert_notes": False
}

modifier = NoteArrayModifier()

def format_data(data: np.ndarray):
    return formatter(data).astype(np.float32).copy()

anchor = data[0]
pos = modifier(anchor, aug_dict)
neg = data[20]
print(anchor)
print(pos)
print(neg)

anchor = format_data(anchor)
pos = format_data(pos)
neg = format_data(data[17])

chosen_models = [pt_model, ft_encoder]

for chosen_model in chosen_models:
    print(f'{chosen_model.__class__.__name__}\n \n \n')
    for i in range(20):
        anch_idx, neg_idx = np.random.choice(len(data), 2, replace=False)
        anchor = data[anch_idx]
        pos = modifier(anchor, aug_dict)
        neg = data[neg_idx]
        anchor = format_data(anchor)
        pos = format_data(pos)
        neg = format_data(neg)
        anchor_enc = chosen_model((torch.from_numpy(anchor)).unsqueeze(0))[0].detach().numpy()   
        pos_enc = chosen_model((torch.from_numpy(pos)).unsqueeze(0))[0].detach().numpy()
        neg_enc = chosen_model((torch.from_numpy(neg)).unsqueeze(0))[0].detach().numpy()

        print(f'pos distance: {np.linalg.norm(anchor_enc - pos_enc)}')
        print(f'neg distance: {np.linalg.norm(anchor_enc - neg_enc)}\n')

C:\Users\cunn2\AppData\Local\Temp\ipykernel_107204\392890555.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(r"C:\Users\cunn2\OneDrive\DSML\Project\th

[[ 0.2 67. ]
 [ 1.  74. ]
 [ 2.  76. ]
 [ 0.8 74. ]]
tensor([[ 0.4000, 67.0000],
        [ 1.0000, 74.0000],
        [ 2.0000, 76.0000],
        [ 0.6000, 74.0000]], dtype=torch.float64)
[[ 0.1 71. ]
 [ 0.5 74. ]
 [ 0.5 67. ]
 [ 0.5 66. ]
 [ 0.5 67. ]
 [ 0.5 69. ]
 [ 1.  71. ]
 [ 0.4 71. ]]
SiameseModel
 
 

pos distance: 3.3639256954193115
neg distance: 7.18888521194458

pos distance: 2.8401143550872803
neg distance: 11.970954895019531

pos distance: 0.3652719259262085
neg distance: 1.0090306997299194

pos distance: 0.07179311662912369
neg distance: 5.881744861602783

pos distance: 2.7604892253875732
neg distance: 6.665331840515137

pos distance: 2.950261116027832
neg distance: 2.7157983779907227

pos distance: 0.421926349401474
neg distance: 4.402246475219727

pos distance: 0.3847900331020355
neg distance: 2.1110687255859375

pos distance: 2.352128744125366
neg distance: 4.557206153869629

pos distance: 3.3295023441314697
neg distance: 6.268095970153809

pos distance: 5.2995357513427